In [1]:
import pandas as pd
import re
from datetime import datetime
import numpy as np

In [2]:
year = 2012
data = pd.read_csv('data/raw-data/espn-raw-pbp-'+str(year)+'.csv')
seasontype = pd.read_csv('data/raw-data/gametypes.csv')

In [3]:
data.columns

Index(['id', 'period', 'text', 'awayScore', 'homeScore', 'clock', 'homeAway',
       'scoringPlay', 'isFoul', 'shootingPlay', 'player_id', 'player',
       'shotType', 'x_coord', 'y_coord', 'homeTeam', 'awayTeam', 'winner',
       'gameID', 'dateTime', 'neutralSite', 'isOT', 'otNum'],
      dtype='object')

In [4]:
#add all of the columns so they are in the same order for all years
list_of_columns = ['EventID','Event', 'EventPlayer', 'Shooter',
       'SpecificShotType', 'ShotValue', 'ShotOutcome',
       'Fouler','FoulType', 'TechnicalType',
       'ReboundType',
       'Assist', 'Blocker', 
       'TurnoverType', 'Steal',
       'SubstitutionOut', 'SubstitutionIn',
       'JumpBallPlayer1', 'JumpBallPlayer2', 'JumpBallPlayerPoss', 
       'TimeoutType', #'StoppageType', 'InstantReplayType',
       'ViolationType']


In [5]:
shot = ['makes','made','miss','misses','missed']
rebound = ['Offensive R','Defensive R','offensive rebound','Offensive Rebound','defensive rebound','Defensive Rebound','offensive team rebound','defensive team rebound']
foul = ['Foul','personal foul','shooting foul','loose ball foul','offensive charge','offensive Charge',
        'flagrant foul type 1','double personal foul','offensive foul',
        'away from play foul','personal take foul','clear path foul',
       'shooting block foul','personal blocking foul','in bound foul',
        'flagrant foul type 2','personal block',
       'Personal Foul','Shooting Foul','Loose Ball Foul','Offensive Foul','Clear Path Foul',
       'Inbound Foul','Flagrant Foul Type 1','Flagrant Foul Type 2','offensiveCharge','Away From Ball Foul','FlagrantFoulType1',
       'away from play personal foul']
tech_foul = ['technical foul','Technical Foul','delay techfoul','taunting technical','flopping technical foul',#'Double technical foul',
             'defensive 3-seconds','Too Many Players Technical','Delay of Game','non unsport techfoul','Non-unsportsmanlike Technical']
turnover = ['turnover','Turnover','lost ball turnover','Out of Bounds','possession lost ball turnover','bad pass','traveling',
              'double dribble turnover','palming turnover','steps out of bounds','out of bounds lost ball','jumpball violation turnover',
            'inbound turnover','back court turnover','5 sec back to the basket turnover','kicked ball turnover','5 sec inbound turnover',
              'shot clock turnover','Shot Clock Turnover','5 second violation','back court violation','disc dribble','3 second violation',
           'Bad Pass','Lost Ball','Traveling','3 second','Double Dribble','Discontinue Dribble','Offensive Goaltending','Backcourt','travelling','swinging elbows turnover',
           'out of bounds turnover','steps out of bounds turnover','Out-of-Bounds Bad Pass Turnover','out of bounds bad pass turnover',
            'double personal turnover','opposite basket turnover','discontinue dribble turnover','8 second','5 second','24 second','10 second','Illegal Pick']
substitution = ['enters the game for']
timeout = ['Timeout','Full timeout','Official timeout','20 Sec. timeout','Full Timeout','20 Second Timeou','20 Sec. Timeout','Official Timeout','full timeout','official timeout']
block = ['blocks','blocked by']
violation = ['delay of game violation','kicked ball violation','jumpball violation',
              'jump ball violation','lane violation','double lane violation','jumpball violation turnover',
            'Jump Ball Violation','Kicked Ball','Defensive Goaltending','Lane Violation','Illegal Defense']
jump_ball = ['vs\.']
ejection = ['ejected']


event2_list = ['assists','steals','Stolen by']

game_events = ['End of the 1st Quarter','End of the 1st quarter','End of the 2nd Quarter','End of the 3rd Quarter','End of the 4th Quarter',
               'End of the 1st Overtime','End of the 2nd Overtime','End of the 3rd Overtime',
               'End of the 1st  Overtime','End of the 1ST  Overtime','End of the 2nd  Overtime','End of the 3rd  Overtime',
               'End of Game','End Game','Start of the 1st Quarter','Start of the 2nd Quarter','Start of the 3rd Quarter','Start of the 4th Quarter',
               'Start of the 1st Overtime','Start of the 2nd Overtime','Start of the 3rd Overtime','End of the 5th Quarter','End of the 6th  Overtime',
               'Start of the 1st Half','End of the 1st Half','Start of the 2nd Half','End of the 2nd Half']

shot_regex = '|'.join(shot)
rebound_regex = '|'.join(rebound)
#foul = '|'.join(foul)
#turnover = '|'.join(turnover)
#substitution = '|'.join(substitution)
#timeout = '|'.join(timeout)
#block = '|'.join(block)
#violation = '|'.join(violation)
#jump_ball = '|'.join(jump_ball)
#
#event2_list = '|'.join(event2_list)
#game_events = '|'.join(game_events)

#instead make them all into one list to match to so then I can use the in operator to check if it's in one of these groups
all_events = [shot,rebound,foul,tech_foul,turnover,substitution,timeout,block,violation,jump_ball,ejection,event2_list,game_events]
all_events = [item for sublist in all_events for item in sublist]
all_events

['makes',
 'made',
 'miss',
 'misses',
 'missed',
 'Offensive R',
 'Defensive R',
 'offensive rebound',
 'Offensive Rebound',
 'defensive rebound',
 'Defensive Rebound',
 'offensive team rebound',
 'defensive team rebound',
 'Foul',
 'personal foul',
 'shooting foul',
 'loose ball foul',
 'offensive charge',
 'offensive Charge',
 'flagrant foul type 1',
 'double personal foul',
 'offensive foul',
 'away from play foul',
 'personal take foul',
 'clear path foul',
 'shooting block foul',
 'personal blocking foul',
 'in bound foul',
 'flagrant foul type 2',
 'personal block',
 'Personal Foul',
 'Shooting Foul',
 'Loose Ball Foul',
 'Offensive Foul',
 'Clear Path Foul',
 'Inbound Foul',
 'Flagrant Foul Type 1',
 'Flagrant Foul Type 2',
 'offensiveCharge',
 'Away From Ball Foul',
 'FlagrantFoulType1',
 'away from play personal foul',
 'technical foul',
 'Technical Foul',
 'delay techfoul',
 'taunting technical',
 'flopping technical foul',
 'defensive 3-seconds',
 'Too Many Players Technica

In [6]:
years = range(2007,2023)
for year in years:
    data = pd.read_csv('data/raw-data/espn-raw-pbp-'+str(year)+'.csv')
    data = pd.merge(data,seasontype)
    df = data.copy()
    for new_column in list_of_columns:
        df[new_column] = ""
    
    df['EventID'] = range(len(df)) #first number the rows so we can sort back into order
    bad_rows = list()
    temp_df_dts = []
    for row in range(len(df['text'])):
        string = df.loc[row,'text']
        if pd.isnull(string):
            continue
        string = string.strip()
        string = re.sub('\. Her.*','',string)
        tracker = 0
        for event_type in all_events:
            if bool(re.search(event_type,string)):
                try: 
                    #print("Match!!:"+event_type)
                    tracker += 1 #track whether we have matched to an event in our list
                    
                    if event_type in shot:
                        df.loc[row,'Event'] = 'Shot'
                        if event_type in ['makes','made']:
                            df.loc[row,'ShotOutcome'] = 'made'
                        if event_type in ['miss','missed','misses']:
                            df.loc[row,'ShotOutcome'] = 'missed'
                        match = re.match(r"(?P<player>[\w\s\-\.';]*)([:\s]+)?(?P<event>"+event_type+")([:\s]+)?(?P<shot_type>[\w\s\-]*)?([:\s]+)?(?P<paren>\([\w\s\-\.';]*\))?(\.)?", string)
                        df.loc[row,'EventPlayer'] = match.group('player').strip('\. ')
                        df.loc[row,'Shooter'] = match.group('player').strip('\. ')
                        df.loc[row,'SpecificShotType'] = match.group('shot_type').strip('\. ')
                        if 'three' in match.group('shot_type'):
                            df.loc[row,'ShotValue'] = '3pt Shot'
                        elif 'free' in match.group('shot_type'):
                            df.loc[row,'ShotValue'] = 'Free Throw'
                        else:
                            df.loc[row,'ShotValue'] = '2pt Shot'
                        
                        
                    if event_type in rebound:
                        df.loc[row,'Event'] = 'Rebound'
                        match = re.match(r"(?P<player>[\w\s\-\.';]*)([:\s]+)?(?P<event>"+event_type+")([:\s]+)?(?P<shot_type>[\w\s\-]*)?([:\s]+)?(?P<paren>\([\w\s\-\.';]*\))?(\.)?.*", string)
                        if 'team' in event_type:
                            df.loc[row,'EventPlayer'] = "Team"
                        else:
                            df.loc[row,'EventPlayer'] = match.group('player').strip('\. ')
                        if bool(re.search('[Oo]ffensive',event_type)):
                            df.loc[row,'ReboundType'] = 'Offensive'
                        if bool(re.search('[Dd]efensive',event_type)):
                            df.loc[row,'ReboundType'] = 'Defensive'
                        
                    if event_type in foul:
                        df.loc[row,'Event'] = 'Foul'
                        match = re.match(r"(?P<player>[\w\s\-\.';]*)([:\s]+)?(?P<event>"+event_type+")(\..*)?", string)
                        df.loc[row,'isFoul'] = True
                        df.loc[row,'FoulType'] = event_type
                        df.loc[row,'Fouler'] = match.group('player').strip('\. ')
                        df.loc[row,'EventPlayer'] = match.group('player').strip('\. ')
                        if bool(re.search('[Dd]ouble '+event_type, string)):
                            if bool(re.search("\(",string)):
                                match = re.match(r"(?P<player>[\w\s\-\.';]*) double personal foul([:\s]+)?\((?P<player2>[\w\s\-\.';]*) draws the foul\)", string)
                            else:
                                match = re.match(r"(?P<player>[\w\s\-\.';]*) Double Personal Foul\. (?P<player2>[\w\s\-\.';]*) gains possession\.", string)
                            df.loc[row,'EventPlayer'] = match.group('player2').strip('\. ')
                            df.loc[row,'Fouler'] = match.group('player2').strip('\. ')
                            df.loc[row,'FoulType'] = 'Double Personal Foul'
                            dtrow = df.iloc[row]
                            temp_df_dts.append(list(dtrow))
                            df.loc[row,'EventPlayer'] = match.group('player').strip('\. ')
                            df.loc[row,'Fouler'] = match.group('player').strip('\. ')
    
                                         
                    if event_type in tech_foul:            
                        df.loc[row,'Event'] = 'Technical Foul'
                        df.loc[row,'TechnicalType'] = event_type
                        df.loc[row,'FoulType'] = ''
                        df.loc[row,'Fouler'] = ''
                        if "Double" in string:
                            if bool(re.search(":",string)):
                                match = re.match(r"Double technical foul: (?P<player>[\w\s\-\.';]*)([:\s]+)? and (?P<player2>[\w\s\-\.';]*)([:\s]+)?(\.)?", string)
                            else:
                                match = re.match(r"(?P<player>[\w\s\-\.';]*) Double Technical Foul\. (?P<player2>[\w\s\-\.';]*) gains possession\.", string)
                            df.loc[row,'EventPlayer'] = match.group('player2').strip('\. ')
                            df.loc[row,'TechnicalType'] = 'Double technical foul'
                            dtrow = df.iloc[row]
                            temp_df_dts.append(list(dtrow))
                            df.loc[row,'EventPlayer'] = match.group('player').strip('\. ')
                        else:
                            string2 = re.sub("([\(\[]).*?([\)\]])", "", string)
                            if event_type in string2:
                                match = re.match(r"(?P<player>[\w\s\-\.';]*)([:\s]+)?(?P<event>"+event_type+").*", string2)
                                df.loc[row,'EventPlayer'] = match.group('player').strip('\. ')
                                         
                    if event_type in turnover:
                        if not(bool(re.search('offensive foul',string))):
                            df.loc[row,'Event'] = 'Turnover'
                            df.loc[row,'TurnoverType'] = event_type
                            match = re.match(r"(?P<player>[\w\s\-\.';]*)([:\s]+)?(?P<event>"+event_type+")(\.)?", string)
                            player = match.group('player')
                            player = re.sub('with a','',player)
                            player = re.sub('called for','',player)
                            df.loc[row,'EventPlayer'] = player.strip('\. ')
                                         
                    if event_type in substitution:
                        df.loc[row,'Event'] = 'Substitution'
                        match = re.match(r"(?P<player>[\w\s\-\.';]*)([:\s]+)?(?P<event>"+event_type+")([:\s]+)?(?P<player2>[\w\s\-\.';]*)([:\s]+)?(\.)?", string)
                        df.loc[row,'SubstitutionOut'] = match.group('player2').strip('\. ')
                        df.loc[row,'SubstitutionIn'] = match.group('player').strip('\. ')
                                         
                    if event_type in timeout:
                        df.loc[row,'Event'] = 'Timeout'
                        df.loc[row,'TimeoutType'] = event_type
                        match = re.match(r"(?P<team>[\w\s\-\.';]*)([:\s]+)?(?P<event>"+event_type+")(\.)?", string)
                        #df.loc[row,'EventTeam'] = match.group('team').strip('\. ')
                        
                    if event_type in block:
                        df.loc[row,'Event'] = 'Block'
                        df.loc[row,'ShotOutcome'] = "blocked"
                        if event_type == 'blocks':
                            match = re.match(r"(?P<player>[\w\s\-\.';]*)([:\s]+)?blocks([:\s]+)?(?P<player2>[\w\s\-\.';]*)?([:\s]+)?'(s)?([:\s]+)?(?P<shot_type>[\w\s\-]*)?(\.)?", string)
                            df.loc[row,'EventPlayer'] = match.group('player').strip('\. ')
                            df.loc[row,'Blocker'] = match.group('player').strip('\. ')
                            df.loc[row,'Shooter'] = match.group('player2').strip('\. ')
                            df.loc[row,'SpecificShotType'] = match.group('shot_type').strip('\. ')
                            if bool(re.search("[Tt]hree",match.group('shot_type'))):
                                df.loc[row,'ShotValue'] = '3pt Shot'
                            else:
                                df.loc[row,'ShotValue'] = '2pt Shot'
                        if event_type == "blocked by":
                            match = re.match(r"(?P<player>[\w\s\-\.';]*)([:\s]+)?'(s)?([:\s]+)?(?P<shot_type>[\w\s\-]*)?([:\s]+)?blocked by([:\s]+)?(?P<player2>[\w\s\-\.';]*)?(\.)?", string)
                            df.loc[row,'EventPlayer'] = match.group('player').strip('\. ')
                            df.loc[row,'Blocker'] = match.group('player2').strip('\. ')
                            df.loc[row,'Shooter'] = match.group('player').strip('\. ')
                            df.loc[row,'SpecificShotType'] = match.group('shot_type').strip('\. ')
                            if bool(re.search("[Tt]hree",match.group('shot_type'))):
                                df.loc[row,'ShotValue'] = '3pt Shot'
                            else:
                                df.loc[row,'ShotValue'] = '2pt Shot'
                        
                    if event_type in violation:
                        df.loc[row,'Event'] = 'Violation'
                        match = re.match(r"(?P<player>[\w\s\-\.';]*)?([:\s]+)?"+event_type, string)
                        if bool(match):
                            df.loc[row,'EventPlayer'] = match.group('player').strip('\. ')
                        df.loc[row,'ViolationType'] = event_type
                        
                    if event_type in jump_ball:
                        df.loc[row,'Event'] = 'Jump Ball'
                        match = re.match(r"(Jumpball: )?(?P<player>[\w\s\-\.';]*)([:\s]+)?(?P<event>"+event_type+")([:\s]+)?(?P<player2>[\w\s\-\.';]*)?([:\s]+)?.*", string)
                        df.loc[row,'JumpBallPlayer1'] = match.group('player').strip('\. ')
                        p2 = re.sub('\..*','',match.group('player2').strip('\. '))
                        df.loc[row,'JumpBallPlayer2'] = p2
                        if bool(re.search('gains possession',string)):
                            match2 = re.match(r".*[\(\.](?P<paren>[\w\s\-\.';]*) gains possession(\s)?[\)\.]", string)
                            df.loc[row,'JumpBallPlayerPoss'] = match2.group('paren').strip('\. ')
                     
                    if event_type in ejection:
                        df.loc[row,'Event'] = 'Ejection'
                        match = re.match(r"(?P<player>[\w\s\-\.';]*)([:\s]+)?(?P<event>"+event_type+")(\.)?", string)
                        df.loc[row,'EventPlayer'] = match.group('player').strip('\. ')
                        
                    if event_type in event2_list:
                        if event_type == "assists":
                            match = re.match(r".*\((?P<paren>[\w\s\-\.';]*)"+event_type+"(\s)?\)(\.)?", string)
                            df.loc[row,'Assist'] = match.group('paren').strip('\. ')
                        if event_type == "steals":
                            match = re.match(r".*\((?P<paren>[\w\s\-\.';]*)"+event_type+"(\s)?\)(\.)?", string)
                            df.loc[row,'Steal'] = match.group('paren').strip('\. ')
                        if event_type == "Stolen by":
                            match = re.match(r".*\.\s"+event_type+"(?P<paren>[\w\s\-\.';]*)(\.)?", string)
                            df.loc[row,'Steal'] = match.group('paren').strip('\. ')
                                            
                    if event_type in game_events:
                        if bool(re.search("End",string)):
                            df.loc[row,'Event'] = 'End of Period'
                        if bool(re.search("Start",string)):
                            df.loc[row,'Event'] = 'Start of Period'
                    
                except:
                    pass
                    #print('Error!  ' + string+ ', row ' + str(row))
                    #print(event_type)
                    
        if tracker == 0:
            bad_rows.append(row)
            if bool(re.search("[Ff]oul",string)):
                match = re.match(r"(?P<player>[\w\s\-\.';]*)(\([\w\s\-\.';]*)?([:\s]+)?[\s]?[Ff]oul(\s\))?", string)
                df.loc[row,'EventPlayer'] = match.group('player').strip('\. ')
                df.loc[row,'Fouler'] = match.group('player').strip('\. ')
                df.loc[row,'Event'] = 'Foul'
                df.loc[row,'isFoul'] = True
            elif bool(re.search("[Vv]iolation",string)):
                df.loc[row,'Event'] = 'Violation'
            elif bool(re.search("[Tt]urnover",string)):
                match = re.match(r"(?P<player>[\w\s\-\.';]*)([:\s]+)?[\s]?[Tt]urnover", string)
                df.loc[row,'EventPlayer'] = match.group('player').strip('\. ')
                df.loc[row,'Event'] = 'Turnover'
            else:
                pass
                #print('No match!  ' + string)
    #need to handle double techs
    temp_df_dts = pd.DataFrame(temp_df_dts, columns=df.columns)
    for row in range(len(temp_df_dts)):
        if temp_df_dts.loc[row,'homeAway'] == 'home':
            temp_df_dts.loc[row,'homeAway'] = 'away'
        elif temp_df_dts.loc[row,'homeAway'] == 'away':
            temp_df_dts.loc[row,'homeAway'] = 'home'
    df = pd.concat([df,temp_df_dts], ignore_index=True)
    
    #save df
    df = df.sort_values('EventID')
    df.to_csv('data/espn-clean-pbp-'+str(year)+'.csv')

/var/folders/2g/lrjr1h8s3nb2gbrhfn5_1bww9yg270/T/ipykernel_22275/1429878848.py:3: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/raw-data/espn-raw-pbp-'+str(year)+'.csv')
/var/folders/2g/lrjr1h8s3nb2gbrhfn5_1bww9yg270/T/ipykernel_22275/1429878848.py:3: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/raw-data/espn-raw-pbp-'+str(year)+'.csv')
/var/folders/2g/lrjr1h8s3nb2gbrhfn5_1bww9yg270/T/ipykernel_22275/1429878848.py:3: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/raw-data/espn-raw-pbp-'+str(year)+'.csv')
/var/folders/2g/lrjr1h8s3nb2gbrhfn5_1bww9yg270/T/ipykernel_22275/1429878848.py:3: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/raw-data/espn-raw-pbp-'+str(year)+

In [7]:
temp_df_dts[['homeAway','homeTeam','awayTeam','EventPlayer','FoulType','TechnicalType']]

,homeAway,homeTeam,awayTeam,EventPlayer,FoulType,TechnicalType
0,home,DAL,IND,Charli Collier,,Double technical foul
1,away,WSH,LV,Dearica Hamby,,Double technical foul
2,home,PHX,SEA,Diana Taurasi,,Double technical foul
3,home,LA,DAL,Chiney Ogwumike,,Double technical foul
4,away,LV,CONN,DeWanna Bonner,,Double technical foul
5,away,DAL,SEA,Ezi Magbegor,Double Personal Foul,
6,away,MIN,IND,NaLyssa Smith,,Double technical foul
7,away,WSH,CONN,Jonquel Jones,,Double technical foul
8,home,NY,ATL,Sabrina Ionescu,,Double technical foul
9,home,LA,PHX,Liz Cambage,,Double technical foul


In [8]:
df

,id,period,text,homeAway,awayScore,homeScore,clock,scoringPlay,shootingPlay,player_id,...,Blocker,TurnoverType,Steal,SubstitutionOut,SubstitutionIn,JumpBallPlayer1,JumpBallPlayer2,JumpBallPlayerPoss,TimeoutType,ViolationType
0,4014304754,1,Jordin Canada vs. Breanna Stewart (Jordin Cana...,away,0,0,10:00,NaN,NaN,NaN,...,,,,,,Jordin Canada,Breanna Stewart,Jordin Canada,,
1,4014304757,1,Epiphanny Prince blocks Jordin Canada 's 14-fo...,away,0,0,9:38,NaN,True,3142250.0,...,Epiphanny Prince,,,,,,,,,
2,4014304759,1,Breanna Stewart defensive rebound,home,0,0,9:35,NaN,NaN,NaN,...,,,,,,,,,,
3,40143047510,1,Breanna Stewart misses 25-foot three point pul...,home,0,0,9:29,NaN,True,2998928.0,...,,,,,,,,,,
4,40143047511,1,Nneka Ogwumike defensive rebound,away,0,0,9:29,NaN,NaN,NaN,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88372,401391865548,4,Veronica Burton defensive rebound,away,116,86,9.4,NaN,NaN,NaN,...,,,,,,,,,,
88373,401391865549,4,Veronica Burton steps out of bounds turnover,away,116,86,9.4,NaN,NaN,NaN,...,,steps out of bounds turnover,,,,,,,,
88374,401391865550,4,Jasmine Walker makes 18-foot jumper (Jordin Ca...,home,116,88,7.0,True,True,4065832.0,...,,,,,,,,,,
88375,401391865552,4,End of the 4th Quarter,NaN,116,88,0.0,NaN,NaN,NaN,...,,,,,,,,,,


In [9]:
#to do: who did the fouler foul? may be impossible to tell on non-shooting fouls
#to do: make sure all games are here & get game info